In [8]:
!bash --login -c "poetry update"

Updating dependencies
Resolving dependencies... (9.0s)

No dependencies to install or update


# Option valuation model based on historical prices

This code assumes `aws configure` has been run in the environment of the notebook.

In [1]:
import optionspricing
import boto3
import helpers
import pandas
from dotenv import load_dotenv

load_dotenv()

BINANCE_ETH = "ETHUSDT"
BINANCE_XRP = "XRPUSDT"
BINANCE_SOL = "SOLUSDT"
BINANCE_MATIC = "MATICUSDT"

BINANCE_SYMBOL = BINANCE_MATIC

USD_QUOTES = ["SOL", "XRP", "MATIC"]
quote_in_usd = len([prefix for prefix in USD_QUOTES if BINANCE_SYMBOL.upper().startswith(prefix)]) > 0

s3 = boto3.resource('s3')

data_file = helpers.fetch_object(s3, "test-binance-prices-255120844515", f"{BINANCE_SYMBOL}-full.csv.zip")
prices_df = pandas.read_csv(data_file, compression='zip', header=0, index_col="dateTime")
prices_df.index = pandas.to_datetime(prices_df.index)

## Option valuation model: input parameters here

In [2]:
target_period_hours = 8 * 24
strikes_universe_size = 3
CUT_OFF_YEAR_MONTH = (2021, 7)

instrument_code = BINANCE_SYMBOL[:-4]

headers = {"Content-Type": "application/json"}
base_url = "https://www.deribit.com/api/v2/public"

trading_model = optionspricing.TradingModel(base_url, headers, instrument_code, target_period_hours)
trading_model.cutoff_year_month(CUT_OFF_YEAR_MONTH)
target_expiry = trading_model.target_expiry
underlying_price = trading_model.underlying_price
remaining_hours = trading_model.remaining_hours

option_chain_df = trading_model.evaluate(prices_df, strikes_universe_size)
simulation = trading_model.simulate_strategy_long_straddle(option_chain_df, strikes_universe_size, quote_in_usd=quote_in_usd)
print(simulation)

ERROR:root:failed to load bid and ask for https://www.deribit.com/api/v2/public/get_order_book_by_instrument_id?instrument_id=341190&depth=1
ERROR:root:failed to load bid and ask for https://www.deribit.com/api/v2/public/get_order_book_by_instrument_id?instrument_id=343253&depth=1
ERROR:root:failed to load bid and ask for https://www.deribit.com/api/v2/public/get_order_book_by_instrument_id?instrument_id=341199&depth=1
ERROR:root:failed to load bid and ask for https://www.deribit.com/api/v2/public/get_order_book_by_instrument_id?instrument_id=341201&depth=1


target expiry: Fri 12 Jul, 08:00 (190 hours left)
current price: 0.5107
-------------------------------
hit ratio: 48%
cost: 0.025 / value: 0.037, gain% = 1.28%
($) cost: 0.02 / value: 0.04, average gain = 0.01
buy put 0.5
buy call 0.54
-------------------------------
hit ratio: 40%
cost: 0.013 / value: 0.025, gain% = 1.13%
($) cost: 0.01 / value: 0.02, average gain = 0.01
buy put 0.48
buy call 0.56
-------------------------------
hit ratio: 28%
cost: 0.007 / value: 0.014, gain% = 0.78%
($) cost: 0.01 / value: 0.01, average gain = 0.01
buy put 0.45
buy call 0.58


In [102]:
simulation.options_chain

,value_call,value_call_pct,call_bid,call_ask,value_put,value_put_pct,put_bid,put_ask
strike,,,,,,,,
3050.0,126.209997,NaN,0.0360,0.0415,9.103572,0.002879,0.0034,0.0041
3075.0,104.305922,NaN,0.0315,0.0345,12.199497,0.003858,0.0045,0.0055
3100.0,83.538378,NaN,0.0230,0.0290,16.431953,0.005197,0.0065,0.0070
3125.0,64.449229,NaN,0.0195,0.0220,22.342804,0.007066,0.0080,0.0090
3150.0,47.901750,NaN,0.0150,0.0165,30.795325,0.009739,0.0110,0.0120
3175.0,35.123000,0.011108,0.0105,0.0120,43.016574,NaN,0.0145,0.0160
3200.0,25.929310,0.008200,0.0075,0.0085,58.822885,NaN,0.0185,0.0215
3225.0,19.162961,0.006060,0.0050,0.0060,77.056535,NaN,0.0225,0.0285
3250.0,14.166475,0.004480,0.0033,0.0041,97.060050,NaN,0.0285,0.0345


# Sanity checks

In [15]:

trading_model = optionspricing.TradingModel(base_url, headers, instrument_code, target_period_hours)
option_chain_df = trading_model.evaluate(prices_df, strikes_universe_size=size)
size = 6
put_weights = [0.] * (2 * size + 1)
call_weights = [0.] * (2 * size + 1)
put_weights[size - 5] = +1.
put_weights[size - 1] = -1.
call_weights[size + 1] = -1.
call_weights[size + 5] = +1.

simulation = trading_model.simulate_strategy(option_chain_df, put_weights, call_weights, quote_in_usd=quote_in_usd)
print(simulation)

target expiry: Fri 24 May, 08:00 (18 hours left)
current price: 0.526
-------------------------------
hit ratio: 0%
cost: nan / value: -0.017, gain% = nan%
($) cost: nan / value: -0.02, average gain = nan
buy put 0.49
sell put 0.52
sell call 0.53
buy call 0.56


In [7]:
trading_model.calls

{(0.3, datetime.datetime(2024, 7, 5, 8, 0)): 339353,
 (0.4, datetime.datetime(2024, 7, 5, 8, 0)): 339361,
 (0.45, datetime.datetime(2024, 7, 5, 8, 0)): 339367,
 (0.5, datetime.datetime(2024, 7, 5, 8, 0)): 339377,
 (0.52, datetime.datetime(2024, 7, 5, 8, 0)): 339381,
 (0.54, datetime.datetime(2024, 7, 5, 8, 0)): 339385,
 (0.55, datetime.datetime(2024, 7, 5, 8, 0)): 339387,
 (0.56, datetime.datetime(2024, 7, 5, 8, 0)): 339389,
 (0.58, datetime.datetime(2024, 7, 5, 8, 0)): 339391,
 (0.6, datetime.datetime(2024, 7, 5, 8, 0)): 339393,
 (0.62, datetime.datetime(2024, 7, 5, 8, 0)): 339395,
 (0.64, datetime.datetime(2024, 7, 5, 8, 0)): 339397,
 (0.65, datetime.datetime(2024, 7, 5, 8, 0)): 339399,
 (0.66, datetime.datetime(2024, 7, 5, 8, 0)): 339341,
 (0.7, datetime.datetime(2024, 7, 5, 8, 0)): 339401,
 (0.75, datetime.datetime(2024, 7, 5, 8, 0)): 339403,
 (0.8, datetime.datetime(2024, 7, 5, 8, 0)): 339347,
 (0.9, datetime.datetime(2024, 7, 5, 8, 0)): 339349,
 (70.0, datetime.datetime(2024, 7, 